In [1]:
%pip install zenml matplotlib
!zenml integration install tensorflow mlflow -y
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

  Using cached packaging-20.9-py2.py3-none-any.whl (40 kB)


  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
statsmodels 0.13.2 requires packaging>=21.3, but you have packaging 20.9 which is incompatible.
label-studio 1.5.0.post0 requires boto3~=1.16.28, but you have boto3 1.24.33 which is incompatible.
label-studio 1.5.0.post0 requires botocore~=1.19.28, but you have botocore 1.27.33 which is incompatible.
label-studio 1.5.0.post0 requires google-api-core==1.31.5, but you have google-api-core 2.8.2 which is incompatible.
label-studio 1.5.0.post0 requires google-auth==1.35.0, but you have google-auth 2.9.1 which is incompatible.
label-studio 1.5.0.post0 requires google-cloud-core==1.5.0, but you have google-cloud-core 2.3.2 which is incompatible.
label-studio 1.5

Note: you may need to restart the kernel to use updated packages.
⠦ Installing integrations.....Requirement already satisfied: tensorflow==2.8.0 in /home/apenner/PycharmProjects/zenml/venv/lib/python3.8/site-packages (2.8.0)
⠧ Installing integrations...Requirement already satisfied: opt-einsum>=2.3.2 in /home/apenner/PycharmProjects/zenml/venv/lib/python3.8/site-packages (from tensorflow==2.8.0) (3.3.0)


⠇ Installing integrations...Requirement already satisfied: gitdb<5,>=4.0.1 in /home/apenner/PycharmProjects/zenml/venv/lib/python3.8/site-packages (from gitpython>=2.1.0->mlflow<1.26.0,>=1.2.0) (4.0.9)
⠏ Installing integrations...Requirement already satisfied: google-auth-oauthlib<0.5,>=0.4.1 in /home/apenner/PycharmProjects/zenml/venv/lib/python3.8/site-packages (from tensorboard<2.9,>=2.8->tensorflow==2.8.0) (0.4.6)
⠋ Installing integrations...Requirement already satisfied: greenlet!=0.4.17 in /home/apenner/PycharmProjects/zenml/venv/lib/python3.8/site-packages (from sqlalchemy->mlflow<1.26.0,>=1.2.0) (1.1.2)
⠹ Installing integrations...Collecting click>=7.0
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
⠸ Installing integrations...Requirement already satisfied: prometheus-client in /home/apenner/PycharmProjects/zenml/venv/lib/python3.8/site-packages (from prometheus-flask-exporter->mlflow<1.26.0,>=1.2.0) (0.14.1)
⠼ Installing integrations...Requirement already satisfied: rs

⠙ Installing integrations...
  Using cached pyparsing-2.4.2-py2.py3-none-any.whl (65 kB)
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
zenml 0.8.0 requires click<9.0.0,>=8.0.1, but you have click 7.1.2 which is incompatible.
azureml-core 1.39.0.post1 requires urllib3<=1.26.7,>=1.23, but you have urllib3 1.26.11 which is incompatible.
apache-airflow 2.2.0 requires jsonschema~=3.0, but you have jsonschema 4.7.2 which is incompatible.
apache-airflow 2.2.0 requires packaging~=21.0, but you have packaging 20.9 which is incompatible.
You should consider upgrading via the '/home/apenner/PycharmProjects/zenml/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to

{'status': 'ok', 'restart': True}

In [2]:
# COLAB ONLY setup
try:
    import google.colab

    IN_COLAB = True

    # clone zenbytes repo to get source code of previous lessons
    !git clone https://github.com/zenml-io/zenbytes.git  # noqa
    !mv examples/mlflow_deployment/steps .
    !mv examples/mlflow_deployment/pipelines .

except ModuleNotFoundError as err:
    IN_COLAB = False

In [3]:
from pipelines import continuous_deployment_pipeline, inference_pipeline
from steps.deployment_trigger.deployment_trigger_step import (
    DeploymentTriggerConfig,
    deployment_trigger,
)
from steps.dynamic_importer.dynamic_importer_step import dynamic_importer
from steps.importer.importer_step import importer_mnist
from steps.normalizer.normalizer_step import normalizer
from steps.prediction_service_loader.prediction_service_loader_step import (
    MLFlowDeploymentLoaderStepConfig,
    model_deployer,
    prediction_service_loader,
)
from steps.predictor.predictor_step import predictor
from steps.tf_evaluator.tf_evaluator_step import tf_evaluator
from steps.tf_predict_preprocessor.tf_predict_preprocessor_step import (
    tf_predict_preprocessor,
)
from steps.tf_trainer.tf_trainer_step import TrainerConfig, tf_trainer

In [4]:
from zenml.integrations.mlflow.mlflow_utils import get_tracking_uri
from zenml.integrations.mlflow.model_deployers.mlflow_model_deployer import (
    MLFlowModelDeployer,
)
from zenml.integrations.mlflow.services import MLFlowDeploymentService
from zenml.integrations.mlflow.steps import MLFlowDeployerConfig

In [8]:
deployment = continuous_deployment_pipeline(
    importer=importer_mnist(),
    normalizer=normalizer(),
    trainer=tf_trainer(config=TrainerConfig(epochs=5, lr=0.001)),
    evaluator=tf_evaluator(),
    deployment_trigger=deployment_trigger(
        config=DeploymentTriggerConfig(
            min_accuracy=0.01,
        )
    ),
    model_deployer=model_deployer(
        config=MLFlowDeployerConfig(workers=3, timeout=10)
    ),
)

deployment.run()

Creating run for pipeline: continuous_deployment_pipeline
Cache enabled for pipeline continuous_deployment_pipeline


WARNING - (py.warnings._showwarnmsg) - /home/apenner/PycharmProjects/zenml/venv/lib/python3.8/site-packages/tensorboard/_vendor/html5lib/filters/sanitizer.py:29: DeprecationWarning:

html5lib's sanitizer is deprecated; see https://github.com/html5lib/html5lib-python/issues/443 and please let us know if Bleach is unsuitable for your needs



html5lib's sanitizer is deprecated; see https://github.com/html5lib/html5lib-python/issues/443 and please let us know if Bleach is unsuitable for your needs




╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_280576/3294074411.py:16 in <cell line: 16>                                        │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_280576/3294074411.py'                       │
│                                                                                                  │
│ /home/apenner/PycharmProjects/zenml/src/zenml/pipelines/base_pipeline.py:481 in run              │
│                                                                                                  │
│   478 │   │   │   schedule=schedule,                                                             │
│   479 │   │   │   **additional_parameters,                                                       │
│   480 │   │   )                                                                                  │
│ ❱ 481 │   │   stack = Repository().active_stack                                                  │
│   482 │   │                                                                                      │
│   483 │   │   stack_metadata = {                                                                 │
│   484 │   │   │   component_type.value: component.FLAVOR                                         │
│                                                                                                  │
│ /home/apenner/PycharmProjects/zenml/src/zenml/repository.py:795 in active_stack                  │
│                                                                                                  │
│    792 │   │   Returns:                                                                          │
│    793 │   │   │   The active stack for this repository.                                         │
│    794 │   │   """                                                                               │
│ ❱  795 │   │   return self.get_stack(name=self.active_stack_name)                                │
│    796 │                                                                                         │
│    797 │   @property                                                                             │
│    798 │   def active_stack_name(self) -> str:                                                   │
│                                                                                                  │
│ /home/apenner/PycharmProjects/zenml/src/zenml/repository.py:853 in get_stack                     │
│                                                                                                  │
│    850 │   │   Returns:                                                                          │
│    851 │   │   │   The stack with the given name.                                                │
│    852 │   │   """                                                                               │
│ ❱  853 │   │   return self.zen_store.get_stack(name).to_stack()                                  │
│    854 │                                                                                         │
│    855 │   def register_stack(self, stack: Stack) -> None:                                       │
│    856 │   │   """Registers a stack and its components.                                          │
│                                                                                                  │
│ /home/apenner/PycharmProjects/zenml/src/zenml/zen_stores/models/stack_wrapper.py:58 in to_stack  │
│                                                                                                  │
│   55 │   │   stack_components = {}                                                               │
│   56 │   │   for component_wrapper in self.components:                                           │
│   57 │   │   │   component_type = component_wrapper.type   